## <div style="text-align: right">   第11章 </div>

##  <div style="text-align: right"> 进程皆可衍生</div>
___

### 11.1 Luke, 使用 _fork(2)_

衍生（forking）是Unix编程中最强大的概念之一。 _fork（2）_ 系统调用允许运行中的进程以编程的方式创建新的进程。这个新进程和原始进程
一模一样。

到目前为止，我们都是通过在终端中运行的方式来创建新的进程。我们也提到了底层操作系统调用 _fork(2)_ 的工作原理。

进行衍生时，调用 _fork(2)_ 的进程被称为“父进程”，新创建的进程被称为“子进程”。

**子进程从父进程继承了其所占用内存的所有内容，以及所有属于父进程的已打开的文件描述符。** 让我们来简单回顾一下从前三章中学到的有关子进程的知识。

因为子进程是一个全新的概念，所以它拥有自己唯一的pid。

子进程的上层进程（parent）显然就是它的父进程。因此子进程的ppid就是调用 _fork(2)_ 的进程的pid。

在 _fork(2)_ 调用时，子进程从父进程处继承了所有的文件描述符，也获得了父进程所有的文件描述符的编号。这样，这两个进程就可以共享
打开的文件、套接字等等。

子进程继承了父进程内存中的所有内容。借助这种方式，一个进程，比如说python，可以将一个500MB的代码库（codebase）装入内存，
然后该进程衍生出两个子进程，这些子进程实际上各自享有一份已载入内存的代码库的副本。

fork调用几乎瞬间完成，这样我们就得到了3个进程，每个进程都可以使用500MB的内存空间。这对于需要在内存中载入多个应用程序实例而言简直就是完美的解决方案。因为只需要一个进程来载入应用程序，而且进程衍生的速度很快，所以这种方法比分别载入3个应用程序实例要快的多。

子进程可以随意更改其内存内容的副本，而不会对父进程造成影响。在下一章中，我们讨论“写时复制（copy on write）技术，以及该技术在进程衍生时如何影响内存内容。

下面通过一个令人费解的例子开始我们在python中的进程衍生之旅。

In [1]:
import os

if os.fork():
    print('entered the if block')
else:
    print('entered the else  block')

entered the if block
entered the else  block


到底怎么回事？对fork方法的调用采用了我们熟悉的if结构，但是结果却是南辕北辙。不知道怎么回事，这段代码竟然同时执行了if语句的if结构和else代码块。

这一切不难理解。对于fork方法的一次调用实际上返回了两次。记住，fork创造了一个新进程。所以它在调用进程（父进程）中返回一次，在新创建的进程（子进程）中又返回一次。

如果我们把pid打印出来的话，上面的例子就更显而易见了。

> 在下述代码示例中，使用os模块的getpid()方法返回当前进程的pid值

In [7]:
print('parent process pid is {}'.format(os.getpid()))

parent process pid is 7539


In [18]:
if os.fork():
    print('entered the if block from {}'.format(os.getpid()))
else:
    print('entered the else block from {}'.format(os.getpid()))

entered the if block from 7539
entered the else block from 11724


现在一切都豁然开朗了：if语句块中的代码是由父进程执行的，而else语句块中的代码是子进程执行的。子进程执行完else语句块后退出，父进程则继续执行。

这种执行顺序可不是乱来的，它和fork方法的返回值有关，在**子进程中，fork返回0**,因为0代表假，所以子进程执行了else语句块中的代码。

在**父进程中，fork返回新创建的子进程的pid**。因为此整数值非0,为真，所以父进程执行的是if语句块中的代码。

只需打印fork调用的值，这个概念便能清晰地展现出来。

In [19]:
print(os.fork())

11765
0


这里我们获得了两个不同的返回值，第一个返回值代表的是新创建的子进程的pid，这个值来自于父进程;第二个返回值则是来自于子进程的0。

### 11.2 多核编程

间接来说，就是这样。通过生成新的进程，你的代码可以（不能完全保证）被分配到多个CPU核心中。

在配备了4个CPU的系统中，如果衍生出4个新进程，那么这些进程会分别由不同的CPU来处理，从而实现多核并发（multicore oncurrency）。

然而，并不保证它们会并行操作。在繁忙的系统中，有可能所有4个进程都由同一个CPU来处理。

_fork(2)_ 创建了一个和旧进程一模一样的新进程。所以如果一个使用了500MB内存的进程进行了衍生，那么就有1GB的内存被占用了。

重复同样的操作10次，你很快就会耗尽所有的内存。这通常称为’fork炸弹（fork bomb）。使用并发执行前，请务必确保你知道这样做的后果。

### 11.3 使用block

在上面的例子中，我们使用if/else结构演示了fork的用法。如果需要部分代码仅在子进程中执行，部分代码仅在父进程中执行，则可通过判断fork方法返回的值来判断当前处于哪个进程。

In [25]:
print("parent process pis is {}".format(os.getpid()))

parent process pis is 7539


In [28]:
pid = os.fork()
if pid > 0:
    # 仅在父进程中运行
    print('running in the parent process, pid is {}'.format(os.getpid()))
else:
    # 仅在子进程中运行
    print('running in the subprocess, pid is {}'.format(os.getpid()))

# 在所有进程中都执行
print('current process pid is {}'.format(os.getpid()))

running in the parent process, pid is 7539
current process pid is 7539
running in the subprocess, pid is 12094
current process pid is 12094


### 11.4 实践领域

请参看附录或Spyglass项目来学习 _fork(2)_ 的实践用法。

### 11.5 系统调用

python中os库的fork方法对应与 _fork(2)_。